# Cohort 8 Task-3 Decompose

Que: Shown using the U and CX, obtaining CCX and CCCX gates. Also, created a method for constructing any multi-controlled X gate.

In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit_ibm_provider import IBMProvider
from qiskit.visualization import array_to_latex
from IPython.core.display import HTML

In [2]:
pi=np.pi

Drawing a simple CCX gate using qiskit built in function

In [3]:
qc =QuantumCircuit(3,3)
qc.ccx(2,1,0)
qc.draw()

┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──■──
       │  
q_2: ──■──
          
c: 3/═════

To decompose this gate to CNot and single qubit unitaries I use the methods given in this paper '<a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>'

<img src="Images/CCX.png" width="600" height="200">

Here V is such $V^2=X$ and U is an X gate

Now we have to create all the above gates used in the circuit using CNOT and Single Qubit Unitary. That shall be given by:

### SX Gate
Sx gate is such that $SX^2=X$

In [4]:
#creating root x gate from u
root_X_circuit=QuantumCircuit(1)
root_X_circuit.u(pi/2,-pi/2,-pi/2,0)#equivalent to qiskit sx gate(that can be used direcctly)
root_x=root_X_circuit.to_gate(label='sx')

### SXDG Gate
Creating conjugate of SX gate

In [5]:
root_X_dagger_circuit=QuantumCircuit(1)
root_X_dagger_circuit.u(-pi/2,-pi/2,-pi/2,0)#equivalent to qiskit sxdg gate(that can be used direcctly)
root_x_dg=root_X_dagger_circuit.to_gate(label='sxdg')

### CSX Gate
lemma 5.1 from <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>

In [6]:
root_X_control_circuit=QuantumCircuit(2)#equivalent to qiskit csx gate(that can be used direcctly)
root_X_control_circuit.u(pi/2,0,pi,1)
root_X_control_circuit.u(0,0,pi/4,0)
root_X_control_circuit.cx(0,1)
root_X_control_circuit.u(0,0,-pi/4,1)
root_X_control_circuit.cx(0,1)
root_X_control_circuit.u(pi/2,0,-3*pi/4,1)
c_root_x=root_X_control_circuit.to_gate(label='csx')#creating csx gate from u and cx gates

### CSXDG Gate

In [7]:
root_X_control_dagger_circuit=QuantumCircuit(2)#dagger of csx gate
root_X_control_dagger_circuit.u(-pi/2,0,pi,1)
root_X_control_dagger_circuit.u(0,0,pi/4,0)
root_X_control_dagger_circuit.cx(0,1)
root_X_control_dagger_circuit.u(0,0,-pi/4,1)
root_X_control_dagger_circuit.cx(0,1)
root_X_control_dagger_circuit.u(-pi/2,0,-3*pi/4,1)
root_X_control_dagger_circuit.u(0,0,-pi/2,0)
c_root_x_dg=root_X_control_dagger_circuit.to_gate(label='csxdg')#creating csxdg gate from u and cx gates

# CCX

Making CCX gate function using aboves gates we get

lemma 6.1 from <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>

In [8]:
#ccx gate
def ccx(qc,qubits):
    #Appends the ccx gate to the given quantum circuit qc
    #Params: qc-QuntumCircuit
    #Params: qc-QuntumCircuit
    #Params: qubits-list : the qubits order in which the gates have to be applied
    #Returns the Final Circuit
    
    qc.append(c_root_x,qubits[-2:])
    qc.cx(qubits[-3],qubits[-2])
    qc.append(c_root_x_dg,qubits[-2:])
    qc.cx(qubits[-3],qubits[-2])
    qc.append(c_root_x,qubits[:-2]+[qubits[-1]])
    return ccx

Drawing the circuit

In [9]:
qc=QuantumCircuit(3)
ccx(qc,[2,1,0])
qc.draw()

┌──────┐     ┌────────┐     ┌──────┐
q_0: ┤1     ├─────┤1       ├─────┤1     ├
     │  csx │┌───┐│  csxdg │┌───┐│      │
q_1: ┤0     ├┤ X ├┤0       ├┤ X ├┤  csx ├
     └──────┘└─┬─┘└────────┘└─┬─┘│      │
q_2: ──────────■──────────────■──┤0     ├
                                 └──────┘

Getting the Unitary for the above circuit to show indeed we have CCX gate

In [10]:
backend = Aer.get_backend('unitary_simulator')
job = backend.run(transpile(qc, backend))
array_to_latex(job.result().get_unitary(qc, decimals=3), prefix='Output = ', max_size=24)

<IPython.core.display.Latex object>

Now similarly we wish to create a CCCX gate with 3 control qubits

lemma 6.1 from <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>

# CCCX Gate

#### The method used for CCCX gate shall use lemma 7 from <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>

We created V such that $V^4=X$ 

The circuit for such V is given by 

<img src="Images/CCCX.png" width="800" height="300">

Creating the above used gates using CNots and U Gate

### CXRoot2 Gate
Creating controlled V gate Such that $V^4=X$ Using lemma 5.1

<img src="Images/CW.png" width="600" height="200">

In [11]:
#creating gate c_root2_x such that (c_root2_x)^4=CX gate.
root2_X_control_circuit=QuantumCircuit(2)
root2_X_control_circuit.u(0,0,pi/2,1)
root2_X_control_circuit.cx(0,1)
root2_X_control_circuit.u(-pi/8,0,0,1)
root2_X_control_circuit.cx(0,1)
root2_X_control_circuit.u(pi/8,-pi/2,0,1)
root2_X_control_circuit.u(0,0,pi/8,0)
c_root2_x=root2_X_control_circuit.to_gate(label='csx2')#creating csx2 gate from u and cx gates

### CXRoot2Dagger Gate
Complex conjugate of the V gate above

In [12]:
#Complex conjugate of c_root2_x gate
root2_X_control_dagger_circuit=QuantumCircuit(2)#equivalent to qiskit csx gate(that can be used direcctly)
root2_X_control_dagger_circuit.u(0,0,-pi/2,1)
root2_X_control_dagger_circuit.cx(0,1)
root2_X_control_dagger_circuit.u(-pi/8,0,0,1)
root2_X_control_dagger_circuit.cx(0,1)
root2_X_control_dagger_circuit.u(pi/8,pi/2,0,1)
root2_X_control_dagger_circuit.u(0,0,15*pi/8,0)
c_root2_x_dg=root2_X_control_dagger_circuit.to_gate(label='csx2_dg')#creating conjugate of csx2 gate from u and cx gates

### CCCX Function

Creating a CCCX function to create a cccx gate using CNot's and Gates prepared above using unitaries


In [13]:
def cccx(qc,qubits):
    #Appends the cccx gate to the given quantum circuit qc
    #Params: qc-QuntumCircuit
    #Params: qubits-list : the qubits order in which the gates have to be applied
    #Returns the Final Circuit 
    
    qc.append(c_root2_x,[qubits[-4]]+[qubits[-1]])
    qc.cx(qubits[-4],qubits[-3])
    qc.append(c_root2_x_dg,[qubits[-3]]+[qubits[-1]])
    qc.cx(qubits[-4],qubits[-3])
    qc.append(c_root2_x,[qubits[-3]]+[qubits[-1]])
    qc.cx(qubits[-3],qubits[-2])
    qc.append(c_root2_x_dg,[qubits[-2]]+[qubits[-1]])
    qc.cx(qubits[-4],qubits[-2])
    qc.append(c_root2_x,[qubits[-2]]+[qubits[-1]])
    qc.cx(qubits[-3],qubits[-2])
    qc.append(c_root2_x_dg,[qubits[-2]]+[qubits[-1]])
    qc.cx(qubits[-4],qubits[-2])
    qc.append(c_root2_x,[qubits[-2]]+[qubits[-1]])
    return qc

In [14]:
qc=QuantumCircuit(4)
cccx(qc,[3,2,1,0])
qc.draw()

┌───────┐     ┌──────────┐     ┌───────┐     ┌──────────┐     ┌───────┐»
q_0: ┤1      ├─────┤1         ├─────┤1      ├─────┤1         ├─────┤1      ├»
     │       │     │          │     │       │┌───┐│  csx2_dg │┌───┐│  csx2 │»
q_1: ┤       ├─────┤  csx2_dg ├─────┤  csx2 ├┤ X ├┤0         ├┤ X ├┤0      ├»
     │  csx2 │┌───┐│          │┌───┐│       │└─┬─┘└──────────┘└─┬─┘└───────┘»
q_2: ┤       ├┤ X ├┤0         ├┤ X ├┤0      ├──■────────────────┼───────────»
     │       │└─┬─┘└──────────┘└─┬─┘└───────┘                   │           »
q_3: ┤0      ├──■────────────────■──────────────────────────────■───────────»
     └───────┘                                                              »
«          ┌──────────┐     ┌───────┐
«q_0: ─────┤1         ├─────┤1      ├
«     ┌───┐│  csx2_dg │┌───┐│  csx2 │
«q_1: ┤ X ├┤0         ├┤ X ├┤0      ├
«     └─┬─┘└──────────┘└─┬─┘└───────┘
«q_2: ──■────────────────┼───────────
«                        │           
«q_3: ───────────────────■───────────
«

Printing the final unitary using unitary simulator

In [15]:
backend = Aer.get_backend('unitary_simulator')
job = backend.run(transpile(qc, backend))
array_to_latex(job.result().get_unitary(qc, decimals=3), prefix='Output = ', max_size=24)

<IPython.core.display.Latex object>

Thus we successfully created a CCCX gate

Now we wish to create a fuction that creates a controlled not gate for any n number of control qubits using a unitary V and its controlled gates such that $V^2=X$ 


# Bonus
lemma 7.5 <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>

Method to create a N controlled U gate

<img src="Images/CNX.png" width="600" height="200">

The ancillia method for n-1 controled V gate comes from

lemma 7.11 <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>


<img src="Images/C(N-1)W.png" width="400" height="100">

I shall really appreciate if someone could help me implement the method without ancillia qubit used to make cnsx gate using lemma 5.1 and lemma 7.5

<img src="Images/CNW.png" width="400" height="100">

### CNX Function

In [19]:
def cnx(qc,n,qubits):
    #Appends the cnx gate to the given quantum circuit qc
    #Params: qc-QuntumCircuit
    #Params: n-Number of control qubits
    #Params: qubits-list : the qubits order in which the gates have to be applied
    #Returns the Final Circuit 
    
    if n==1:
        qc.cx(qubits[-2],qubits[-1])
        return qc
    elif n==2:
        ccx(qc,qubits)
    elif n==3:
        cccx(qc,qubits)
    else:
        
        num_qubits=qc.num_qubits
        qc.reset(ar[0])
        qc.append(c_root_x,qubits[-2:])
        cnx(qc,n-1,qubits[1:-1])
        qc.append(c_root_x_dg,qubits[-2:])
        cnx(qc,n-1,qubits[1:-1])
        cnx(qc,n-1,qubits[:-2][::-1])
        qc.append(c_root_x,[qubits[0]]+[qubits[-1]])
        cnx(qc,n-1,qubits[:-2][::-1])
        
    return qc

This method for created CNX gates require an extra Ancilla Qubit that must be provided while invoking the function

In [20]:
from qiskit.circuit import QuantumRegister,ClassicalRegister,AncillaRegister
qr=QuantumRegister(5)
cr=ClassicalRegister(5)
ar=AncillaRegister(1,'an')
qc=QuantumCircuit(qr,cr,ar)
cnx(qc,4,[5,4,3,2,1,0])
qc.draw()

┌──────┐                                                              »
q9_0: ┤1     ├──────────────────────────────────────────────────────────────»
      │  csx │┌───────┐     ┌──────────┐     ┌───────┐     ┌──────────┐     »
q9_1: ┤0     ├┤1      ├─────┤1         ├─────┤1      ├─────┤1         ├─────»
      └──────┘│       │     │          │     │       │┌───┐│  csx2_dg │┌───┐»
q9_2: ────────┤       ├─────┤  csx2_dg ├─────┤  csx2 ├┤ X ├┤0         ├┤ X ├»
              │  csx2 │┌───┐│          │┌───┐│       │└─┬─┘└──────────┘└─┬─┘»
q9_3: ────────┤       ├┤ X ├┤0         ├┤ X ├┤0      ├──■────────────────┼──»
              │       │└─┬─┘└──────────┘└─┬─┘└───────┘                   │  »
q9_4: ────────┤0      ├──■────────────────■──────────────────────────────■──»
              └───────┘                                                     »
  an: ──|0>─────────────────────────────────────────────────────────────────»
                                                                            »
c1: 5/══════════════════════════════════════════════════════════════════════»
                                                                            »
«                                              ┌────────┐              »
«q9_0: ────────────────────────────────────────┤1       ├──────────────»
«      ┌───────┐     ┌──────────┐     ┌───────┐│  csxdg │┌───────┐     »
«q9_1: ┤1      ├─────┤1         ├─────┤1      ├┤0       ├┤1      ├─────»
«      │  csx2 │┌───┐│  csx2_dg │┌───┐│  csx2 │└────────┘│       │     »
«q9_2: ┤0      ├┤ X ├┤0         ├┤ X ├┤0      ├──────────┤       ├─────»
«      └───────┘└─┬─┘└──────────┘└─┬─┘└───────┘          │  csx2 │┌───┐»
«q9_3: ───────────■────────────────┼─────────────────────┤       ├┤ X ├»
«                                  │                     │       │└─┬─┘»
«q9_4: ────────────────────────────■─────────────────────┤0      ├──■──»
«                                                        └───────┘     »
«  an: ────────────────────────────────────────────────────────────────»
«                                                                      »
«c1: 5/════════════════════════════════════════════════════════════════»
«                                                                      »
«                                                                    »
«q9_0: ──────────────────────────────────────────────────────────────»
«      ┌──────────┐     ┌───────┐     ┌──────────┐     ┌───────┐     »
«q9_1: ┤1         ├─────┤1      ├─────┤1         ├─────┤1      ├─────»
«      │          │     │       │┌───┐│  csx2_dg │┌───┐│  csx2 │┌───┐»
«q9_2: ┤  csx2_dg ├─────┤  csx2 ├┤ X ├┤0         ├┤ X ├┤0      ├┤ X ├»
«      │          │┌───┐│       │└─┬─┘└──────────┘└─┬─┘└───────┘└─┬─┘»
«q9_3: ┤0         ├┤ X ├┤0      ├──■────────────────┼─────────────■──»
«      └──────────┘└─┬─┘└───────┘                   │                »
«q9_4: ──────────────■──────────────────────────────■────────────────»
«                                                                    »
«  an: ──────────────────────────────────────────────────────────────»
«                                                                    »
«c1: 5/══════════════════════════════════════════════════════════════»
«                                                                    »
«                                                                             »
«q9_0: ───────────────────────────────────────────────────────────────────────»
«      ┌──────────┐     ┌───────┐                                             »
«q9_1: ┤1         ├─────┤1      ├─────────────────────────────────────────────»
«      │  csx2_dg │┌───┐│  csx2 │┌───────┐                                    »
«q9_2: ┤0         ├┤ X ├┤0      ├┤0      ├──■────────────────■────────────────»
«      └──────────┘└─┬─┘└───────┘│       │┌─┴─┐┌──────────┐┌─┴─┐┌───────┐     »
«q9_3: ──────────────┼───────────┤       ├┤ X ├┤0         ├┤ X ├┤0      ├──■──»
«                    │           │  

Printing the matrix for the above Circuit

In [18]:
job = backend.run(transpile(qc, backend))
array_to_latex(job.result().get_unitary(qc, decimals=3), max_size=64)


<IPython.core.display.Latex object>

We see that this matrix differs from a CNOT, that is due to it including the Ancilla qubit in the simulation. In use it shall act as cnx gate for the non ancilla qubits provided or maybe I have made a mistake somewhere. I am looking for methods that shall allow me to write this unitary without incling the ancilla. Any insight shall be appriciated

Special thanks to people who made this work possible aka the contributors to this paper <a href="https://arxiv.org/pdf/quant-ph/9503016.pdf">Elementary gates for quantum computation</a>  

Thankyou QOSF for the opportunity to work on this mentorship program